<a href="https://colab.research.google.com/github/ekaterinatao/house_md_tg_bot/blob/main/notebooks/NLP_2_HW1_house_md_bot_data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [ ]:
import re
from glob import glob
from tqdm import tqdm
import pandas as pd
from datasets import Dataset

import warnings
warnings.filterwarnings("ignore")

# Загрузка данных с kaggle
Исходные данные на [kaggle](https://www.kaggle.com/datasets/kunalbhar/house-md-transcripts/code)

In [ ]:
!pip install kaggle -q
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download kunalbhar/house-md-transcripts
!unzip house-md-transcripts.zip

  0% 0.00/2.15M [00:00<?, ?B/s]
100% 2.15M/2.15M [00:00<00:00, 119MB/s]
Archive:  house-md-transcripts.zip
  inflating: season1.csv             
  inflating: season2.csv             
  inflating: season3.csv             
  inflating: season4.csv             
  inflating: season5.csv             
  inflating: season6.csv             
  inflating: season7.csv             
  inflating: season8.csv             


In [ ]:
# Загрузка реплик всех сезонов
data_raw = pd.concat(
    [
        pd.read_csv(i, encoding='unicode_escape')
        .assign(season=re.findall('[1-9]', i)[0]) for i in glob('season[1-8].csv')
    ],
    axis=0
).reset_index(drop=True)
data_raw

,name,line,season
0,Inmate,"Deep fried shrimp, and lobster. I nEver had l...",2
1,Lawyer,We'll do our best to accommodate. Tomorrow yo...,2
2,Inmate,"Naw, I don't need none of that.",2
3,Lawyer,One last thing to think about: After I read t...,2
4,Inmate,"Yo, Clarence! You hear that? A spiritual advi...",2
...,...,...,...
75307,Cuddy,Are you okay?,5
75308,Amber,So! This is the story you made up about who y...,5
75309,Kutner,Too bad it isn't true.,5
75310,House,[shakily as Cuddy touches his cheek] No. I'm ...,5


In [ ]:
data_raw['name'].value_counts(normalize=True).head(10)

House       0.277622
Foreman     0.087504
Chase       0.066245
Wilson      0.064718
Cameron     0.055105
Cuddy       0.050683
Taub        0.042451
Thirteen    0.026171
Kutner      0.015084
House       0.012150
Name: name, dtype: float64

In [ ]:
def preprocess_text(data: pd.DataFrame) -> pd.DataFrame:
    'Удаляем пояснения,"эмоциональные" вставки, пустые реплики'
    for ind, row in data.iterrows():
        try:
            row['line'] = re.sub(r"\[.+\]", '', row['line']).strip()
            row['line'] = re.sub(r"\(.+\)", '', row['line']).strip()
        except:
            pass
    data = data[data['line'] != '']
    data = data.dropna().reset_index(drop=True)
    return data

data_raw = preprocess_text(data_raw)
data_raw

,name,line,season
0,Megan Bradberry,"Yeah, I said I'm sorry. I'm just not feeling t...",4
1,BEN PROSNER,You're sneezing. You can do that at a movie th...,4
2,Megan Bradberry,"Yeah, and I'll be in a nice old chair when I c...",4
3,BEN PROSNER,"Oh, come on! We're gonna be late! Episode Four...",4
4,Megan Bradberry,Go without me.,4
...,...,...,...
74783,House,Just switched the dental records.,8
74784,Wilson,You're destroying your entire life. You can't ...,8
74785,House,"I'm dead, Wilson. How do you want to spend you...",8
74786,Wilson,When the cancer starts getting really bad!,8


Т.к. данные не были изначально разбиты на эпизоды, на данном этапе решено воздержаться от подобной предобработки.  
Была взята небольшая длина контекста (равная 3 предшествующим репликам).

In [ ]:
def prepare_data(data: pd.DataFrame, character: str, label: int) -> pd.DataFrame:
    'Отбор для реплики персонажа вопроса и контекста до максимальной длины 3'
    data_new = pd.DataFrame()
    for ind, row in tqdm(data.iterrows()):
        question = []
        if character in row['name']:
            for i in range(1, 4):
                if ind-i >= 0:
                    question.append(data.iloc[ind - i]["line"])
                else:
                    break

            question.reverse()
            for i in range(3):
                new_row = {
                    "answer": row["line"],
                    "question": [question[-1]],
                    "context": question[i:-1],
                    "labels": label
                }
                data_new = data_new.append(new_row, ignore_index=True)

    return data_new

# собираем данные для каждой реплики (ответа) Др.Хауса
data = prepare_data(data=data_raw, character='House', label=0)
data

74788it [04:36, 270.50it/s]


,answer,question,context,labels
0,I'm thinking the broken bones are a response t...,[Two surgeries for multiple fractures and burns.],"[Call 911., Twenty-six-year-old female, gas ma...",0
1,I'm thinking the broken bones are a response t...,[Two surgeries for multiple fractures and burns.],"[Twenty-six-year-old female, gas main exploded...",0
2,I'm thinking the broken bones are a response t...,[Two surgeries for multiple fractures and burns.],[],0
3,Put her on antipyretics.,[And the fEver? She's the only collapse victim...,[Two surgeries for multiple fractures and burn...,0
4,Put her on antipyretics.,[And the fEver? She's the only collapse victim...,[I'm thinking the broken bones are a response ...,0
...,...,...,...,...
65083,"I'm dead, Wilson. How do you want to spend you...",[You're destroying your entire life. You can't...,[Just switched the dental records.],0
65084,"I'm dead, Wilson. How do you want to spend you...",[You're destroying your entire life. You can't...,[],0
65085,Cancer's boring.,[When the cancer starts getting really bad!],[You're destroying your entire life. You can't...,0
65086,Cancer's boring.,[When the cancer starts getting really bad!],"[I'm dead, Wilson. How do you want to spend yo...",0


In [ ]:
#В качестве негативный примеров - реплики (ответы) Кэмерон
data_neg = prepare_data(data=data_raw, character='Cameron', label=1)
data_neg

74788it [00:28, 2663.92it/s] 


,answer,question,context,labels
0,Three weeks. For someone who nEver misses some...,[Actually it is. It's just not your dream title.],[I can't hire you as a doctor. But you can sti...,1
1,Three weeks. For someone who nEver misses some...,[Actually it is. It's just not your dream title.],[Assistant sounds marginally less demeaning.],1
2,Three weeks. For someone who nEver misses some...,[Actually it is. It's just not your dream title.],[],1
3,"The hair, where I'm working, or both?",[You're an idiot.],[Actually it is. It's just not your dream titl...,1
4,"The hair, where I'm working, or both?",[You're an idiot.],[Three weeks. For someone who nEver misses som...,1
...,...,...,...,...
12412,"You know it's the same, or you wouldn't be bic...","[If I kept it to myself, then it would just be...",[What's your point? That you cared about him m...,1
12413,"You know it's the same, or you wouldn't be bic...","[If I kept it to myself, then it would just be...",[],1
12414,But somewhere in there! he knew how to love.,[He wasn't always easy to deal with.],"[He made me a better parent, whether he meant ...",1
12415,But somewhere in there! he knew how to love.,[He wasn't always easy to deal with.],[He was willing to kill me. And I'll always be...,1


In [ ]:
data = pd.concat([data, data_neg]).reset_index(drop=True)
data.shape

(77505, 4)

In [ ]:
def join_tokens(raw):
    'Объединение контекста, вопроса и ответа.'
    new_raw = raw['context'] + ['[SEP]'] + raw['question'] + ['[SEP]'] + [raw['answer']]
    return ' '.join(tok for tok in new_raw)

data['joined'] = data.apply(join_tokens, axis=1)
data

,answer,question,context,labels,joined
0,I'm thinking the broken bones are a response t...,[Two surgeries for multiple fractures and burns.],"[Call 911., Twenty-six-year-old female, gas ma...",0,"Call 911. Twenty-six-year-old female, gas main..."
1,I'm thinking the broken bones are a response t...,[Two surgeries for multiple fractures and burns.],"[Twenty-six-year-old female, gas main exploded...",0,"Twenty-six-year-old female, gas main exploded ..."
2,I'm thinking the broken bones are a response t...,[Two surgeries for multiple fractures and burns.],[],0,[SEP] Two surgeries for multiple fractures and...
3,Put her on antipyretics.,[And the fEver? She's the only collapse victim...,[Two surgeries for multiple fractures and burn...,0,Two surgeries for multiple fractures and burns...
4,Put her on antipyretics.,[And the fEver? She's the only collapse victim...,[I'm thinking the broken bones are a response ...,0,I'm thinking the broken bones are a response t...
...,...,...,...,...,...
77500,"You know it's the same, or you wouldn't be bic...","[If I kept it to myself, then it would just be...",[What's your point? That you cared about him m...,1,What's your point? That you cared about him mo...
77501,"You know it's the same, or you wouldn't be bic...","[If I kept it to myself, then it would just be...",[],1,"[SEP] If I kept it to myself, then it would ju..."
77502,But somewhere in there! he knew how to love.,[He wasn't always easy to deal with.],"[He made me a better parent, whether he meant ...",1,"He made me a better parent, whether he meant t..."
77503,But somewhere in there! he knew how to love.,[He wasn't always easy to deal with.],[He was willing to kill me. And I'll always be...,1,He was willing to kill me. And I'll always be ...


Загружаем датасет на [huggingface](https://huggingface.co/datasets/ekaterinatao/house_md_context3)

In [ ]:
dataset = Dataset.from_pandas(data.reset_index(drop=True))
dataset

Dataset({
    features: ['answer', 'question', 'context', 'labels', 'joined'],
    num_rows: 77505
})

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
dataset.push_to_hub("ekaterinatao/house_md_context3", private=False)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/78 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/ekaterinatao/house_md_context3/commit/ff77d50d74994f697d9471cb500edb83afb979a0', commit_message='Upload dataset', commit_description='', oid='ff77d50d74994f697d9471cb500edb83afb979a0', pr_url=None, pr_revision=None, pr_num=None)